# Combine Model

In [1]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [2]:
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [3]:
import json
import pandas as pd
import numpy as np
import requests
from PIL import Image

In [4]:
model_transfer = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

87916544/87910968 [==============================] - 14s 0us/step


In [5]:
# define combined model
# !!! keep the top layers consistent with the custom model
x = model_transfer.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.8)(x)
x = Dense(10, activation="softmax")(x)
model_combined = Model(model_transfer.input, x)

In [7]:
# load custom layer weights

model.load_weights('model_4.h5')  # update this with the most up-to-date weights, 'model' is the custom model

# get weights for inception v3 and for custom layers
inceptionv3_weights = model_transfer.get_weights()
top_layer_weights = model.get_weights()

# combine weights
combined_weights = inceptionv3_weights + top_layer_weights

# set weights for combined model
model_combined.set_weights(combined_weights)

In [8]:
print(len(inceptionv3_weights))
print(len(top_layer_weights))
print(len(combined_weights))

376
4
380


### Continue training combined model

In [9]:
# get data from json
PATH = "/mnt/disk1/"
with open(PATH + "data/caltech_images_20190919.json", 'r') as f:
    annotations = json.load(f)

annot_df = pd.DataFrame.from_dict(annotations['annotations'])
image_df = pd.DataFrame.from_dict(annotations['images'])

# get animal name map 
category_id = []
category_name = []

for i in annotations['categories']:
    category_id.append(i['id'])
    category_name.append(i['name'])
    
category_dict = dict(zip(category_id, category_name))
annot_df['category_name'] = annot_df['category_id'].map(category_dict)

# drop duplicate annotations
annot_df.sort_values("image_id", inplace = True) 
annot_df_nodup = annot_df.drop_duplicates(subset ="image_id", keep = 'first')

# merge annotation and image data together
image_df.rename(columns={"id": "image_id"}, inplace=True)
image_df.sort_values("image_id", inplace = True) 
image_df_annot = pd.concat([image_df.set_index('image_id'), annot_df_nodup.set_index('image_id')], axis=1, join='inner').reset_index()

image_df_annot.head()

,image_id,seq_num_frames,date_captured,seq_id,height,width,location,rights_holder,file_name,frame_num,id,category_id,category_name
0,5858bf1e-23d2-11e8-a6a3-ec086b02610b,1,2014-07-10 14:20:20,6fc5ac4a-5567-11e8-b3db-dca9047ef277,1494,2048,96,Erin Boydston,test/5858bf1e-23d2-11e8-a6a3-ec086b02610b.jpg,1,52acca15-7d6d-11e7-884d-7845c41c2c67,30,empty
1,5858bf20-23d2-11e8-a6a3-ec086b02610b,1,2014-05-26 13:44:00,6fbb3c73-5567-11e8-b378-dca9047ef277,1494,2048,96,Erin Boydston,test/5858bf20-23d2-11e8-a6a3-ec086b02610b.jpg,1,f77c64a8-7d6e-11e7-884d-7845c41c2c67,30,empty
2,5858bf21-23d2-11e8-a6a3-ec086b02610b,1,2014-09-16 08:16:51,7006f717-5567-11e8-b4c6-dca9047ef277,1494,2048,114,Erin Boydston,test/5858bf21-23d2-11e8-a6a3-ec086b02610b.jpg,1,1aa83e59-7d71-11e7-884d-7845c41c2c67,10,rabbit
3,5858bf22-23d2-11e8-a6a3-ec086b02610b,1,2014-05-27 14:57:44,6fbd60e3-5567-11e8-a122-dca9047ef277,1494,2048,96,Erin Boydston,test/5858bf22-23d2-11e8-a6a3-ec086b02610b.jpg,1,45913ae8-7d6d-11e7-884d-7845c41c2c67,30,empty
4,5858bf23-23d2-11e8-a6a3-ec086b02610b,1,2014-06-06 12:31:06,6fbf5a47-5567-11e8-9655-dca9047ef277,1494,2048,96,Erin Boydston,test/5858bf23-23d2-11e8-a6a3-ec086b02610b.jpg,1,5f7e6e7a-7d6e-11e7-884d-7845c41c2c67,30,empty


In [10]:
classes = ['rodent','squirrel','rabbit','bird','deer','raccoon','skunk','opossum']
classes_dict = dict(zip(classes + ['other'] + ['empty'], range(10)))
classes_dict_lookup = dict(zip(range(10), classes + ['other'] + ['empty']))

orig_class = list(image_df_annot['category_name'].unique())
new_class = [animal if animal in classes + ['empty'] else 'other' for animal in orig_class]
prediction_class_map = dict(zip(orig_class, new_class))

prediction_class_map

{'empty': 'empty',
 'rabbit': 'rabbit',
 'bobcat': 'other',
 'squirrel': 'squirrel',
 'raccoon': 'raccoon',
 'coyote': 'other',
 'bird': 'bird',
 'car': 'other',
 'opossum': 'opossum',
 'cow': 'other',
 'skunk': 'skunk',
 'dog': 'other',
 'deer': 'deer',
 'fox': 'other',
 'cat': 'other',
 'rodent': 'rodent',
 'mountain_lion': 'other',
 'lizard': 'other',
 'badger': 'other',
 'insect': 'other',
 'pig': 'other',
 'bat': 'other'}

In [11]:
image_df_annot['category_name_model'] = image_df_annot['category_name'].map(prediction_class_map)
image_df_annot['category_id_model'] = image_df_annot['category_name_model'].map(classes_dict)
image_df_annot = image_df_annot.sample(frac=1, random_state=1)

In [12]:
image_df_annot.groupby('category_name_model').image_id.count().sort_values()

category_name_model
skunk         1892
rodent        4272
squirrel      4432
bird          9521
raccoon      10981
deer         12191
rabbit       12345
opossum      16634
other        45087
empty       125745
Name: image_id, dtype: int64

In [13]:
model_combined.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

In [14]:
def load_image(path):
    img = Image.open(path)
    img = img.resize(size=(299,299))
    img = np.array(img)
    img = img / 255.0
    # Convert 2-dim gray-scale array to 3-dim RGB array.
    # if (len(img.shape) == 2):
    #     img = np.repeat(img[:, :, np.newaxis], 3, axis=2)
    return img

In [15]:
def generator(X_data, y_data, batch_size):
    i = 0
    while True:
        X_batch = []
        y_batch = []
        
        for b in range(batch_size):
            if i >= len(X_data):
                i = 0
                
            X = X_data[i]
            y = y_data[i]
            X = load_image(X)
            X_batch.append(X)
            y_batch.append(y)
            i += 1
            
        yield np.array(X_batch), np.array(y_batch)

In [16]:
class_dist = dict(image_df_annot.groupby('category_id_model').image_id.count().sort_values())
total = sum(class_dist.values())
class_dist_inverse = {k: total/v for k, v in class_dist.items()}

In [17]:
class_dist_inverse

{6: 128.48837209302326,
 0: 56.905430711610485,
 1: 54.851083032490976,
 3: 25.533032244512132,
 5: 22.13823877606775,
 4: 19.940940037732755,
 2: 19.692183070068854,
 7: 14.614644703619094,
 8: 5.391798079268969,
 9: 1.9332776651159091}

In [18]:
# download suggested split from data website
url = 'https://lilablobssc.blob.core.windows.net/caltechcameratraps/CaltechCameraTrapsSplits_v0.json'
split = requests.get(url).json()

# use suggested split for test
train_val_loc = sorted(list(set(split['splits']['train'])))
test_locs = sorted(list(set(split['splits']['val'])))


# furthur split training and validation
#test_loc = shuffle(test_locs, random_state=2)
test_loc = test_locs[:20]
val_loc = test_locs[20:]

data_train = image_df_annot[image_df_annot['location'].isin(train_val_loc)]
data_val = image_df_annot[image_df_annot['location'].isin(val_loc)]
data_test = image_df_annot[image_df_annot['location'].isin(test_loc)]

print(data_train.shape)
print(data_val.shape)
print(data_test.shape)

# get image vector
img_vector_train = list(PATH + 'data/cct_images/' + data_train['image_id'] + '.jpg')
img_vector_val = list(PATH + 'data/cct_images/' + data_val['image_id'] + '.jpg')
img_vector_test = list(PATH + 'data/cct_images/' + data_test['image_id'] + '.jpg')

# get category vector
cat_name_vector_train = list(data_train['category_name_model'])
cat_id_vector_train = list(data_train['category_id_model'])

cat_name_vector_val = list(data_val['category_name_model'])
cat_id_vector_val = list(data_val['category_id_model'])

cat_name_vector_test = list(data_test['category_name_model'])
cat_id_vector_test = list(data_test['category_id_model'])

(180690, 15)
(28684, 15)
(33726, 15)


In [19]:
# test on 1024 images
sample_x_train = img_vector_train[:1024]
sample_y_train = cat_id_vector_train[:1024]
sample_x_val = img_vector_val[:64]
sample_y_val = cat_id_vector_val[:64]

In [20]:
def create_model():
    model_transfer = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
    x = model_transfer.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.8)(x)
    x = Dense(10, activation="softmax")(x)
    model_combined = Model(model_transfer.input, x)
    model_combined.set_weights(combined_weights)
    return model_combined

In [21]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    parallel_model = create_model()
    parallel_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

parallel_model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensor

In [ ]:
# without reducing empty class, 10 class with adjusted class weights
# train full model incl. inception v3 layers

#X_train = sample_x_train
#y_train = sample_y_train
#X_val = sample_x_val
#y_val = sample_y_val

X_train = img_vector_train
y_train = cat_id_vector_train
X_val = img_vector_val
y_val = cat_id_vector_val

EPOCH = 20
BATCH_SIZE = 128
TRAIN_STEPS = len(X_train)//BATCH_SIZE
VAL_STEPS = len(y_train)//BATCH_SIZE

train_generator = generator(X_train, y_train, BATCH_SIZE)
val_generator = generator(X_val, y_val, BATCH_SIZE)

for i in range(EPOCH):
    print('---------- Now training epoch '+ str(i)+' ---------------')
    model_combined.fit_generator(generator=train_generator, \
                        validation_data=val_generator,\
                        validation_steps=VAL_STEPS, \
                        steps_per_epoch=TRAIN_STEPS, \
                        epochs=1, verbose=1,\
                        class_weight=class_dist_inverse)
                        #use_multiprocessing=True)
    model_combined.save('./weights/modelv_8/model_' + str(i) + '.h5')

---------- Now training epoch 0 ---------------
Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1411 steps, validate for 1411 steps
1343/1411 [===========================>..] - ETA: 8:19 - loss: 15.3658 - accuracy: 0.4124